<a href="https://colab.research.google.com/github/SakshiGehani/J015_NLPassignments/blob/ClassWork/CW4_SentimentAnalysisTwitterData_J015_17_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment Analysis using NLTK and TextBlob



In [0]:
import pandas as pd
#df = pd.read_csv("https://github.com/zfz/twitter_corpus/blob/master/full-corpus.csv")

In [2]:
from google.colab import files
uploaded = files.upload()

Saving full-corpus.csv to full-corpus.csv


In [0]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['full-corpus.csv']))
# Twitter Dataset is now stored in a Pandas Dataframe

In [0]:
## Amazon Order Reviews Dataset
df1 = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)

In [5]:
df2.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [6]:
df1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


## Polarity Score using TextBlob

For a single string

In [7]:
from textblob import TextBlob

print(TextBlob(df2['TweetText'][0]).sentiment[0])        ##Polarity score for 1st row of Twitter dataset
print(TextBlob(df1['reviewText'][0]).sentiment[0])        ##Polarity score for 1st row of Amazon dataset

0.0
0.18000000000000002


For entire column

In [8]:
## Twitter dataset
sen_pol = df2['TweetText'].apply(lambda x:(TextBlob(x).sentiment[0]))
df2["Polarity"] = sen_pol
df2.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Polarity
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000


In [9]:
## Amazon dataset
sen_pol = df1['reviewText'].apply(lambda x:(TextBlob(x).sentiment[0]))
df1.insert(5, 'revTxt_Polarity', sen_pol)
df1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,revTxt_Polarity,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",0.180000,5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,0.170939,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,0.204091,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,0.214217,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,0.350000,5,Still the best,1375574400,"08 4, 2013"


#### Predicting sentiment using Polarity score

In [10]:
def sentiment_textBlob(num):
  if (num > 0): sen = "positive"
  elif (num < 0): sen = "negative"
  else: sen = "neutral"
  return sen

## Twitter dataset
sen = df2['Polarity'].apply(sentiment_textBlob)
df2["Sentiment_TextBlob"] = sen
df2.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Polarity,Sentiment_TextBlob
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,positive


In [11]:
df2.Sentiment.value_counts()

neutral       2333
irrelevant    1689
negative       572
positive       519
Name: Sentiment, dtype: int64

In [15]:
'''for i in range(0, df2.shape[0]):
  if df2.Sentiment[i] =='irrelevant':
    df2.Sentiment[i] = 'neutral'
    
df2.Sentiment.value_counts()'''

## Remove the rows with sentiment = irrelevant 
ind = df2[ df2['Sentiment'] == 'irrelevant' ].index
df2.drop(ind , inplace=True)
df2.shape
df2.Sentiment.value_counts()

neutral     2333
negative     572
positive     519
Name: Sentiment, dtype: int64

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print("Confusion Matrix for Twitter dataset: \n", confusion_matrix(df2['Sentiment'], df2['Sentiment_TextBlob']), end='\n')
print(classification_report(df2['Sentiment'], df2['Sentiment_TextBlob']))

Confusion Matrix for Twitter dataset: 
 [[ 234  171  167]
 [ 201 1213  919]
 [  38  124  357]]
              precision    recall  f1-score   support

    negative       0.49      0.41      0.45       572
     neutral       0.80      0.52      0.63      2333
    positive       0.25      0.69      0.36       519

    accuracy                           0.53      3424
   macro avg       0.52      0.54      0.48      3424
weighted avg       0.67      0.53      0.56      3424



In [17]:
# Amazon dataset
sen = df1['revTxt_Polarity'].apply(sentiment_textBlob)
df1.insert(6, "revTxt_Sentiment_TextBlob", sen)
df1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,revTxt_Polarity,revTxt_Sentiment_TextBlob,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",0.180000,positive,5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,0.170939,positive,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,0.204091,positive,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,0.214217,positive,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,0.350000,positive,5,Still the best,1375574400,"08 4, 2013"


## Sentiment Analysis using NLTK

For single string

In [18]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def sentIntens(text):
  score = 0.0
  for sentence in text.split('.'):
    print(sentence)
    ss = sid.polarity_scores(sentence)
    score += ss['compound']
    for k in (ss):
      print('{0}: {1}, '.format(k, ss[k]), end='\n')
    print()

  print("Sentiment score = ", score)
  print(sentiment_textBlob(score))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [19]:
sentiment = sentIntens(df2.TweetText[0])
sentiment

Now all @Apple has to do is get swype on the iphone and it will be crack
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 

 Iphone that is
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 

Sentiment score =  0.0
neutral


For entire column

In [20]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def sentIntens(text):
  score = 0.0
  for sentence in text.split('.'):
    ss = sid.polarity_scores(sentence)
    score += ss['compound']                 ## 'compound' value refers to the cummulative score that gives an idea of how postive or negative the sentence or string is
    
  return(sentiment_textBlob(score))
  

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [21]:
## Twitter dataset
df2["Sentiment_NLTK"] = df2['TweetText'].apply(sentIntens)
df2.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Polarity,Sentiment_TextBlob,Sentiment_NLTK
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,neutral,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,positive,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,positive,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,positive,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,positive,positive


In [22]:
print("Confusion Matrix for Twitter dataset: \n", confusion_matrix(df2['Sentiment'], df2['Sentiment_NLTK']), end='\n')
print(classification_report(df2['Sentiment'], df2['Sentiment_NLTK']))

Confusion Matrix for Twitter dataset: 
 [[ 297  132  143]
 [ 310 1216  807]
 [  52   93  374]]
              precision    recall  f1-score   support

    negative       0.45      0.52      0.48       572
     neutral       0.84      0.52      0.64      2333
    positive       0.28      0.72      0.41       519

    accuracy                           0.55      3424
   macro avg       0.53      0.59      0.51      3424
weighted avg       0.69      0.55      0.58      3424



In [23]:
## Amazon dataset
sen = df1['reviewText'].apply(sentIntens)
df1.insert(7, "revTxt_Sentiment_NLTK", sen)
df1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,revTxt_Polarity,revTxt_Sentiment_TextBlob,revTxt_Sentiment_NLTK,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",0.180000,positive,positive,5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,0.170939,positive,positive,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,0.204091,positive,positive,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,0.214217,positive,positive,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,0.350000,positive,positive,5,Still the best,1375574400,"08 4, 2013"
